아래 첫번째 링크를 시도를 해보았으나, 2.0.0 의 TFS 버전은 지원을 하지 않음.
TFS_VERSION = 1.3 으로 해서 Endpoint 생성은 되었으나 아래와 같은 에러 발생

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "{"error": "{ \"error\": \"Servable not found for request: Latest(saved_model)\" }"}". See https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/bert-tweet-endpoint2020-07-24-05-09-54 in account 057716757052 for more information.


Performing batch inference with TensorFlow Serving in Amazon SageMaker# inference on Docker Image

https://aws.amazon.com/blogs/machine-learning/performing-batch-inference-with-tensorflow-serving-in-amazon-sagemaker/

SageMaker TensorFlow Serving Container
- https://github.com/aws/sagemaker-tensorflow-serving-container


In [44]:
import boto3
import sagemaker
import pandas as pd
import tensorflow as tf
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

## Repackage Model Artifact
####  Download the Model to the Notebook

In [45]:
%store -r training_job_name

In [46]:
model_download = 'model'
model_repackage = 'model/package'
model_repackage_code = 'model/package/code'
os.makedirs(model_download, exist_ok=True)
os.makedirs(model_repackage, exist_ok=True)
os.makedirs(model_repackage_code, exist_ok=True)

In [47]:
model_download_path = 's3://{}/{}/output/model.tar.gz'.format(bucket,training_job_name) 
model_upload_model_path = 's3://{}/{}/upload-model/model.tar.gz'.format(bucket,training_job_name) 


In [48]:
!aws s3 cp  {model_download_path} {model_download}/model.tar.gz

download: s3://sagemaker-us-west-2-057716757052/tensorflow-training-2020-07-23-13-02-11-499/output/model.tar.gz to model/model.tar.gz


In [49]:
!tar -xvzf   {model_download}/model.tar.gz -C {model_download}

transformers/
transformers/fine-tuned/
transformers/fine-tuned/config.json
transformers/fine-tuned/tf_model.h5
tensorflow/
tensorflow/saved_model/
tensorflow/saved_model/0/
tensorflow/saved_model/0/saved_model.pb
tensorflow/saved_model/0/assets/
tensorflow/saved_model/0/variables/
tensorflow/saved_model/0/variables/variables.index
tensorflow/saved_model/0/variables/variables.data-00001-of-00002
tensorflow/saved_model/0/variables/variables.data-00000-of-00002
tensorflow/
tensorflow/saved_model/
tensorflow/saved_model/0/
tensorflow/saved_model/0/variables/
tensorflow/saved_model/0/variables/variables.index
tensorflow/saved_model/0/variables/variables.data-00000-of-00002
tensorflow/saved_model/0/variables/variables.data-00001-of-00002
tensorflow/saved_model/0/assets/
tensorflow/saved_model/0/saved_model.pb
transformers/
transformers/fine-tuned/
transformers/fine-tuned/tf_model.h5
transformers/fine-tuned/config.json


In [50]:
%%bash -s {model_download} {model_repackage} {model_repackage_code}

cp  requirements.txt $3
cp  inference.py $3
cp -r $1/tensorflow/saved_model/0/ $2
cd $2
tar -czvf model.tar.gz code 0


code/
code/inference.py
code/requirements.txt
0/
0/assets/
0/saved_model.pb
0/variables/
0/variables/variables.index
0/variables/variables.data-00001-of-00002
0/variables/variables.data-00000-of-00002


In [51]:
repackage_model_file = os.path.join(model_repackage, 'model.tar.gz')
print(repackage_model_file)

model/package/model.tar.gz


In [52]:

inference_image = container

In [64]:
import numpy as np
import os
import sagemaker
from sagemaker import get_execution_role
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

sagemaker_session = sagemaker.Session()

prefix = 'tensorflow-training-2020-07-23-13-02-11-499/upload-model'

tfs_version = '1.13'
processor_type = 'cpu'

# model_url = model_upload_model_path
model_url = "s3://{}/model.tar.gz".format(os.path.join(bucket, prefix))
model_name = "bert-tweet" + timestamp_prefix
endpoint_cfg_name = "bert-tweet-endpoint-config" + timestamp_prefix
endpoint_name = "bert-tweet-endpoint" + timestamp_prefix

container = "520713654638.dkr.ecr.{}.amazonaws.com/sagemaker-tensorflow-serving:{}-{}".format(
    region, tfs_version, processor_type)

print('Region:\t{}'.format(region))
#print('Role:\t{}'.format(role))
print(container)

Region:	us-west-2
520713654638.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tensorflow-serving:1.13-cpu


In [65]:
%%bash -s "$role" "$region" "$bucket" "$prefix" "$model_url" "$model_name" "$endpoint_cfg_name" "$endpoint_name" "$container"
# For convenience, we pass in variables in first Python set-up cell

timestamp() {
  date +%Y-%m-%d-%H-%M-%S
}

ROLE_ARN=$1
REGION=$2
BUCKET=$3
# PREFIX=$4
MODEL_DATA_URL=$5
MODEL_NAME=$6
ENDPOINT_CONFIG_NAME=$7
ENDPOINT_NAME=$8
CONTAINER=$9

aws s3 cp model/package/model.tar.gz $MODEL_DATA_URL

aws sagemaker create-model \
    --model-name $MODEL_NAME \
    --primary-container Image=$CONTAINER,ModelDataUrl=$MODEL_DATA_URL \
    --execution-role-arn $ROLE_ARN
    
VARIANT_NAME="TFS"
INITIAL_INSTANCE_COUNT=1
#INSTANCE_TYPE="ml.p2.xlarge"
INSTANCE_TYPE="ml.c4.xlarge"
aws sagemaker create-endpoint-config \
    --endpoint-config-name $ENDPOINT_CONFIG_NAME \
    --production-variants VariantName=$VARIANT_NAME,ModelName=$MODEL_NAME,InitialInstanceCount=$INITIAL_INSTANCE_COUNT,InstanceType=$INSTANCE_TYPE

aws sagemaker create-endpoint \
    --endpoint-name $ENDPOINT_NAME \
    --endpoint-config-name $ENDPOINT_CONFIG_NAME

upload: model/package/model.tar.gz to s3://sagemaker-us-west-2-057716757052/tensorflow-training-2020-07-23-13-02-11-499/upload-model/model.tar.gz
{
    "ModelArn": "arn:aws:sagemaker:us-west-2:057716757052:model/bert-tweet2020-07-24-05-09-54"
}
{
    "EndpointConfigArn": "arn:aws:sagemaker:us-west-2:057716757052:endpoint-config/bert-tweet-endpoint-config2020-07-24-05-09-54"
}
{
    "EndpointArn": "arn:aws:sagemaker:us-west-2:057716757052:endpoint/bert-tweet-endpoint2020-07-24-05-09-54"
}


## Invoke the endpoint without custom Docker image

In [66]:
# tweet_bert_endpoint_name = 'sagemaker-tensorflow-serving-2020-07-09-00-37-50-247'
tweet_bert_endpoint_name = endpoint_name
print(tweet_bert_endpoint_name)

bert-tweet-endpoint2020-07-24-05-09-54


In [67]:
tweet_bert_endpoint_name

'bert-tweet-endpoint2020-07-24-05-09-54'

In [68]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name = tweet_bert_endpoint_name,
                      sagemaker_session = sess,
                      content_type = 'application/json',
                      model_name = 'saved_model',
                      model_version=0
                     )

In [69]:
# reviews = ["This is not great!", "You are awesome because you look great"]
# reviews = ["The weather is so nice causing me to be depressed!"]
reviews = ["hi"]
predicted_classes = predictor.predict(reviews)
predicted_classes[0]

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "{"error": "{ \"error\": \"Servable not found for request: Latest(saved_model)\" }"}". See https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/bert-tweet-endpoint2020-07-24-05-09-54 in account 057716757052 for more information.

In [ ]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    print(top_n_idx)


    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    print(top_n_idx_list)
    print(top_n_values)

topN = 3    
show_top_N_label(predicted_classes[0], topN)


## Run on Boto3 Client

In [ ]:
invoke_endpoint = 'sagemaker-tensorflow-serving-2020-07-09-07-31-25-739'

In [ ]:
%%bash
echo {$invoke_endpoint}

In [ ]:
%%bash
aws sagemaker-runtime invoke-endpoint \
  --endpoint-name sagemaker-tensorflow-serving-2020-07-09-07-31-25-739 \
  --body '["This is great"]' \
  --content-type application/json \
  --accept application/json \
  results
echo $results

In [ ]:
payload = "This is great"
payload = '["' + payload + '"]'
print(payload)

In [ ]:
def try_test(num1):
    try:
        sum = num1 + 10
        return sum
    except:
        print("Error occured")
        return num1
    
result = try_test("2")
result

In [ ]:
import json
import os
import io
import boto3
import csv

ENDPOINT_NAME = os.environ['ENDPOINT_NAME'] 
runtime = boto3.client('runtime.sagemaker')


def lambda_handler(event, context):
    # TODO implement
try:
print("Received event: " + json.dumps(event, indent=2))

data = json.loads(json.dumps(event))
payload = data['tweet']
payload = '["' + payload + '"]'
print("Final input: " ,payload)

print(payload)

response = runtime.invoke_endpoint(EndpointName = ENDPOINT_NAME,
                            ContentType = 'application/json',
                            Body = payload)

print(response)
result = json.loads(response['Body'].read().decode())
print(result)
return result
except:
print("An error occurred")
return data


# return {
#     'statusCode': 200,
#     'body': json.dumps('Hello from Lambda!')
# }
